Привет! Меня зовут Исмаилов Исмаил, я буду проверять твой проект :) Можешь обращаться ко мне на «ты». Если тебе комфортно, то и я буду к тебе так обращаться, если нет, то обязательно скажи об этом.

Пожалуйста, не удаляй мои комментарии, которые я буду оставлять в работе. Это особенно поможет, если твои проекты будут отправлены на повторную проверку. При повторных проверках у комментариев будут приписки: "Вторая итерация", "Третья итерация" и т.д. 

Ты тоже можешь реагировать на мои комментарии, но в таком случае постарайся, чтобы твои комментарии отличались от моих: например, выделяй их своим любимым цветом — так у нас не возникнет путаницы :)

Кстати, про цвета! Мои комментарии будут в следующей цветовой гамме: зелеными, желтыми и красными. Например:

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
В случае, если решение на отдельном шаге является полностью правильным. </div>

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
В случае, если решение может стать еще лучше с некоторыми корректировками </div>
 
<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
В случае, если решение какого-то шага требует значительных изменений. Проект не может быть принят с первого раза, если ревью содержит комментарии, помеченные этим цветом </div>

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 

Не стоит иметь ячейки с кодом еще до вступления

# Цели исследования

Изучить данные об играх, найти закономерности, на основе полученных результатов определиться с релкамными компаниями и стратегиями развития оналайн-магазина на следующий год.

# Основные этапы исследования

1. Изучить данные
2. Провести их подготовку, предобработку и очистку
3. Провести исследовательский анализ и обнаружить полезные для бизнеса закономерности
4. Составить портреты пользователей трех исследуемых регионов, для эффективного разделения рекламных компаний
5. Предложить стратегии и направления развития бизнеса на следующий год

In [93]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy import stats as st
import numpy as np

# Первый взгляд на данные

Первые пять строк датасета и общая информация.

In [94]:
try: 
    games_data = pd.read_csv('games.csv')
except:
     games_data = pd.read_csv('/datasets/games.csv')
    
display(games_data.head())
games_data.info()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Приведение всех названий столбцов к строчному регистру

In [95]:
games_data.columns = games_data.columns.str.lower()

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Здорово, что используешь str.lower

# Предобработка данных

## Работа с пропусками и типами данных

Видно, что в датасете есть пропуски.

В столбце ```name``` и ```genre``` их ровно по 2. Видимо при выгрузке данных могла потеряться информация о названии игры и как следствие жанр тоже не выгрузился - возможно эти данные хранятся где-то в виде связанных пар данных. Посмотрим на эти строки.

In [96]:
games_data.query('name.isna()')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Эти строки достаточно похожи, год и платформа совпадают, очень низкие продажи и отсутсвуют данные об оценках и рейтинге. Эти строки, можно удалить, их всего 2. Пропуски в столбцах ```name``` и ```genre``` вероятно связаны между собой. Потерялась информация о названии - потерялся и жанр. Вероятно просто ошибка при выгрузке данных.

In [97]:
games_data = games_data.dropna(subset=['name', 'genre'])

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b> 

Правильно, не стоит бояться удалять данные с пропусками, если их совсем мало

Изучим данные за весь 1993 год и отсортируем по возростанию по столбцу ```na_sales```.

In [98]:
display(games_data.query('year_of_release == 1993').sort_values('na_sales', ascending=False).head(10))
print('Строк в срезе:', len(games_data.query('year_of_release == 1993')))

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
56,Super Mario All-Stars,SNES,1993.0,Platform,5.99,2.15,2.12,0.29,NaN,NaN,NaN
476,Star Fox,SNES,1993.0,Shooter,1.61,0.51,0.80,0.07,NaN,NaN,NaN
892,Mortal Kombat II,SNES,1993.0,Fighting,1.48,0.39,0.00,0.06,NaN,NaN,NaN
1263,Sonic CD,SCD,1993.0,Platform,1.00,0.36,0.09,0.05,NaN,NaN,NaN
1009,Disney's Aladdin,SNES,1993.0,Platform,0.94,0.34,0.21,0.27,NaN,NaN,NaN
1010,Kirby's Adventure,NES,1993.0,Platform,0.79,0.14,0.80,0.02,NaN,NaN,NaN
2102,Tetris 2,NES,1993.0,Puzzle,0.62,0.13,0.21,0.02,NaN,NaN,NaN
1750,Mega Man X,SNES,1993.0,Platform,0.57,0.08,0.50,0.01,NaN,NaN,NaN
852,Super Street Fighter II,SNES,1993.0,Fighting,0.52,0.16,1.29,0.03,NaN,NaN,NaN
2027,Final Fight 2,SNES,1993.0,Action,0.39,0.12,0.49,0.02,NaN,NaN,NaN


Строк в срезе: 60


In [99]:
games_data.query('year_of_release == 1993').sort_values('na_sales', ascending=False).isna().sum()

name                0
platform            0
year_of_release     0
genre               0
na_sales            0
eu_sales            0
jp_sales            0
other_sales         0
critic_score       60
user_score         60
rating             60
dtype: int64

Для всего 1993 года отсутсвуют данные об оценках и рейтинге. Может есть какие-то конкретные года, информация о которых была утеряна.

In [100]:
games_data.sort_values('year_of_release')['year_of_release'].unique()

array([1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987., 1988.,
       1989., 1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997.,
       1998., 1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006.,
       2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015.,
       2016.,   nan])

Все года идут по-порядку без пропусков, значит пропуски в данном столбце - это не конкретный пропущенный год или года, а разные значения разбросанные по всему диапазону. Корректно восстановить такие такие пропуски не получится. А учитывая, что их не так много, можно просто избавиться от этих строк.

In [101]:
games_data = games_data.dropna(subset=['year_of_release'])

Столбец с обозначением года выхода игры ```year_of_release``` имеет тип *float*, но номер года это целое число без десятичной части. Изменим тип данных в этом столбце на *int*.

In [102]:
games_data = games_data.astype({'year_of_release' : 'int64'})

Попробуем понять природу пропусков, посмотрим как как распределены пропуски по годам

In [103]:
year_of_release_data = games_data.set_index('year_of_release').notna()

year_of_release_data = year_of_release_data.pivot_table(index=year_of_release_data.index, 
                                 values=['name', 'critic_score', 'user_score', 'rating'], 
                                 aggfunc='sum'
                                )
#year_of_release_data.query('critic_score == 0 or rating == 0 or user_score == 0')
year_of_release_data

,critic_score,name,rating,user_score
year_of_release,,,,
1980,0,9,0,0
1981,0,46,0,0
1982,0,36,0,0
1983,0,17,0,0
1984,0,14,0,0
1985,1,14,1,1
1986,0,21,0,0
1987,0,16,0,0
1988,1,15,1,1


Видно, что до 1995 года данных об оценках и рейтинге практически нет. Вероятно это связано с малым числом игр, выходивших в то время, низким уровнем интереса к индустрии и сложностями сбора оценок игроков. Заполнять такие пропуски какими-либо значениями не корректно, это по сути создаст несуществующую картину оценок для этих годов.

Можно лишь заполнить пропуски в категориальном столбце ```rating```. Заменим пропуски на ```'unknown'```.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b> 

Согласен с названными тобою причинами пропусков

In [104]:
games_data.loc[games_data['rating'].isna(), 'rating'] = 'unknown'

Изменим тип данных столбца ```user_score``` с *object* на *float*. Из задания известно, что в нем есть строковые значения ```'tbd'```. Из глобальной сети интернет узнаем, что такая аббревиатура в геймерских кругах означает **"to be determine"** - **"будет определено"**. Т.е. данных об оценке игроков на данный момент нет. С ними можно работать как с пропусками. Посмотрим на строки с такими данными

In [105]:
display(games_data.query('user_score == "tbd"').head(5))
print('Строк с аббревиатурой "tbd" в столбце "user_score":', round((len(games_data.query("user_score == 'tbd'")) / len(games_data) * 100), 2), '%')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
119,Zumba Fitness,Wii,2010,Sports,3.45,2.59,0.0,0.66,NaN,tbd,E
301,Namco Museum: 50th Anniversary,PS2,2005,Misc,2.08,1.35,0.0,0.54,61.0,tbd,E10+
520,Zumba Fitness 2,Wii,2011,Sports,1.51,1.03,0.0,0.27,NaN,tbd,T
645,uDraw Studio,Wii,2010,Misc,1.65,0.57,0.0,0.20,71.0,tbd,E
718,Just Dance Kids,Wii,2010,Misc,1.52,0.54,0.0,0.18,NaN,tbd,E


Строк с аббревиатурой "tbd" в столбце "user_score": 14.45 %


<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

tbd содержательно означают то же самое, что и nan, поэтому и обработаны должны быть одинаково

Удалить все такие строки не получится, это приведет к потере почти 15% датасета.

Можно отметить особенность таких строк - значения в столбцах с количеством проданных копий относительно низки, если сравнить их с первыми пятью строками всего датасета. Возможно из-за недостаточного объема продаж данных об оценке пользователей недостаточно.

Обычно игра может выпускаться сразу на несколько платформ, можно заполнить такие ```'tbd'```-значения средней оценкой для данной игры высчитанной по оценкам для остальных платформ.

In [106]:
for name in games_data.query('user_score == "tbd"')['name'].unique():
    games_data.loc[(games_data['user_score'] == 'tbd') & (games_data['name'] == name), 
                   'user_score'] = pd.to_numeric(games_data.query('name == @name')['user_score'], 
                                                 errors='coerce').mean()

Теперь преобразуем все данные в столбце ```user_score``` в тип *float*, данные, которые преобразовать не выйдет (например оставшиеся ```'tbd'```-значения, если среднее для игры посчитать не удалось) принудительно отбросим оставив пропуски на их месте.

In [107]:
try:
    games_data['user_score'] = pd.to_numeric(games_data['user_score'])
except:
    games_data['user_score'] = pd.to_numeric(games_data['user_score'], errors='coerce')
    print('Сработала ветка "except"')

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b> 
    
Хвалю за использование конструкции try-except

Отбросить все пустые значения в столбцах ```user_score``` и ```critic_score``` не выйдет, тогда потеряется больше половины всего датасета.

Проверим на наличие полных дубликатов.

In [108]:
games_data.duplicated().sum()

0

Полных дубликатов нет.

Снова посмотрим на общую информацию о датасете.

In [109]:
games_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   float64
 9   user_score       8192 non-null   float64
 10  rating           16444 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


От всех пропусков избавились, а все столбцы имеют соответствующий тип данных.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b> 
    
Здорово, что есть перепроверка

# Добавляем новые столбцы

Добавим столбец с общими продажами по всем регионам.

In [110]:
games_data['total_sales'] = (games_data['na_sales'] + 
                             games_data['eu_sales'] + 
                             games_data['jp_sales'] + 
                             games_data['other_sales']
                            )

In [111]:
#games_data.pivot_table(index='year_of_release', values='name', aggfunc='count').reset_index()

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

other sales отсутствует, японские продажи дважды

# Исследовательский анализ

## Выпуск игр по годам

Построим гистограмму с данными о числе выпущенных игр по годам.

In [112]:
fig = px.bar(games_data.pivot_table(index='year_of_release', values='name', aggfunc='count').reset_index(), 
             x='year_of_release', 
             y='name', 
             height=400, 
             width=900)
fig.show()

Заметно, что число игр, выпускавшихся до 1991 года значительно уступает всем следующим годам. Посчитаем сколько игр было выпущено до 1991 года.

In [113]:
print(f'{games_data.query("year_of_release < 1991")["platform"].count()/len(games_data) : .2%}')

 1.34%


Не больше 2%.

Изучим число игр выпускавшихся на разных платформах

In [114]:
fig = px.histogram(games_data.pivot_table(index='platform', 
                                          values='name', 
                                          aggfunc='count').reset_index().sort_values('name'), 
                   y='platform', 
                   x='name', 
                   height=600, 
                   width=900)
fig.show()

Заметно, что есть ряд платформ, на которые игры выпускались единицами, запомним их в списке ```unpopular_platforms```.

In [115]:
unpopular_platforms = list(games_data.pivot_table(index='platform', 
                                          values='name', 
                                          aggfunc='count').reset_index().sort_values('name').query('name <= 20')['platform'])

In [116]:
games_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16444 non-null  object 
 1   platform         16444 non-null  object 
 2   year_of_release  16444 non-null  int64  
 3   genre            16444 non-null  object 
 4   na_sales         16444 non-null  float64
 5   eu_sales         16444 non-null  float64
 6   jp_sales         16444 non-null  float64
 7   other_sales      16444 non-null  float64
 8   critic_score     7983 non-null   float64
 9   user_score       8192 non-null   float64
 10  rating           16444 non-null  object 
 11  total_sales      16444 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 1.6+ MB


<a id='total_sales_heat_map'></a>

## Исследование проданных копий

Построим тепловую карту где по оси x указаны годы, по оси y указаны платформы, а цвет обозначает число проданных копий.

In [117]:
fig = px.density_heatmap(games_data, 
                         x='year_of_release', 
                         y='platform', 
                         z='total_sales', 
                         nbinsx=40, 
                         height=800, 
                         width=1050, 
                         color_continuous_scale='Viridis', 
                         text_auto=True)
fig.update_layout(margin=dict(l=0, r=0, t=25, b=0))
fig.show()

Вперед к [корреляциям](#correlations).  (Потом пригодится)

По тепловой карте заметно, что популярность платформ ограничена 5-10 годами,  выделяется по продолжительсности использования PC - больше 20 лет.

Также можно заметить 7 абсолютно непопулярных платформ. Продажи игр на этих платформах настолько низки, что они практически не выделяются цветом на тепловой карте. Вероятно это те же платформы, на которые практически не выпускалось игр.

Полученные данные позволяют сделать вывод, что до 1991 года игр выпускалось очень мало, данные до этого года мало чем помогут спрогнозировать 2017 год, их можно отбросить. Заметное изменение происходит после 2011 года, такой период и возьмем.

In [118]:
games_data = games_data.query('year_of_release > 2011')

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Для целей прогнозирования продаж на следующий год даже в традиционных бизнесах редко берут данные более чем за 2-3 года. А в такой динамично меняющейся индустрии, как компьютерные игры и вовсе не стоит брать слишком большой временной интервал - иначе обязательно захватишь уже отжившие тренды. Но и слишком короткий период тоже брать не стоит

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b> 

Все еще слишком большой срок

Также можно отбросить данные по тем самым непопулярным платформам.

In [119]:
games_data = games_data.query('platform not in @unpopular_platforms')

Посмотрим на 10 самых популярных платформ по общим продажам.

In [120]:
popular_platforms = list(games_data.pivot_table(index='platform', 
                                       values='total_sales', 
                                       aggfunc='sum').sort_values('total_sales', 
                                                                  ascending=False).head(10).index
                        )
px.density_heatmap(games_data.query('platform in @popular_platforms'), 
                   x='year_of_release', 
                   y='platform', 
                   z='total_sales',
                   height=500, 
                   color_continuous_scale='Viridis',
                   title='10 наиболее популярных платформ по числу проданных копий игр'
                  )

Часть лидеров по числу продаж - это платформы, которые к 2016 году уже потеряли популярность или почти потеряли - они нас не интересуют. Нужно посмотреть на "живые" платформы.

Можно заметить на какие платформы продаются игры в 2016 году, это:
* PSV (хотя продажи снижаются)
* XOne
* 3DS (тут тоже пик числа проданных копий уже пройден)
* PS4 (наиболее перспективная платформа)
* PC (продажи на момент 16 года низки, но это самая живучая платформа, и ничего не не указывает на то, что в 2017 году PC-гейминг исчезнет)

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Согласен с выбором платформ

## Исследование числа выпущенных игр

Изучим не число проданных копий, а число выпущенных игр.

In [138]:
px.density_heatmap(games_data, 
                   x='year_of_release', 
                   y='platform',
                   width=700,
                   height=400, 
                   color_continuous_scale='Viridis', 
                   text_auto=True
                  )

Картина не сильно изменилась, но теперь видно, что число выпускаемых игр не отражает на-прямую число проданнхы копий. Например для PS4 на момент 2016 года число выпускаемых игр уверенно растет. Аналогичная ситуация, но менее выражена для XOne и PC.

Таким образом к потенциально прибыльным платформам можно отнести PS4, XOne, PC. В меньшей мере к таким платформам относятся те, что все еще "живы" в 2016 году, но у которых снижаются и число проданных копий и число выпускаемых игр, это PSV, 3DS, PS3.

In [123]:
potential_platforms = ['PS4', 'XOne', 'PC', 'PCV', '3DS', 'PS3']

Построим ящики с усами по общим продажам для каждой платформы.

In [124]:
for platform in games_data['platform'].unique():
    fig = px.box(games_data.query('platform == @platform'), 
                 x='total_sales', 
                 y='platform', 
                 title='Платформа ' + platform, 
                 height=200)
    fig.update_layout(margin=dict(l=0, r=0, t=25, b=0))
    fig.show()

Самая продаваемая игра выходила на PS3 - более 20 млн. проданных копий. На популярных платформах есть игры, выделяющиеся на общем фоне по чилу проданных копий - на диаграмме размаха они попадают в категорию выбросов. Это нормально, далеко не каждая игра является успешной.

Интересны платформы, у которых есть значительное число игр проданных тиражом более 10 млн копий - такие игры попадают в число выбросов, так как не являются нормой для индустрии и достаточно редки, но интересны с точки зрения продаж. Определим платформы, на которых вышло хотя бы по одной игре продавшейся тиражом более 10 млн. копий:

 <a id='perspective_platforms'></a>

In [125]:
successful_platforms = list(games_data.query('total_sales > 10').
                            pivot_table(index='platform', values='name', aggfunc='count').
                            query('name >= 1').
                            reset_index()['platform'].
                            values
                           )
print('Платфрмы с 4 и более успешными играми ', successful_platforms)
print('Потенциально привлектельные платформы ', potential_platforms)
print()
print('Наиболее перспективные платформы:')
for platform in successful_platforms:
    if platform in potential_platforms:
        print('-', platform)

Платфрмы с 4 и более успешными играми  ['3DS', 'PS3', 'PS4', 'X360']
Потенциально привлектельные платформы  ['PS4', 'XOne', 'PC', 'PCV', '3DS', 'PS3']

Наиболее перспективные платформы:
- 3DS
- PS3
- PS4


Отлично, у потенциально интересных платформа и платформ с успешными играми есть пересечения. Это развивающиеся или популярные платформы у которых уже есть успешные игры.

## Зависимось продаж от отзывов

<a id='correlations'></a> 

С помощью [тепловой карты по всем продажам](#total_sales_heat_map) выберем одну из наиболее популярных платформ для анализа зависимости продаж от отзывов. Возьмем PC, несмотря на то, что игры на этой платформе не распродаются рекордными тиражами, она остается популярной уже более 20 лет. Нет оснований полагать, что эта платформа исчезнет.

Построим диаграмму рассеяния для отзывов пользователей и критиков для PC.

In [126]:
def two_scatter_plots(platform):   #функция для построения двух диаграмм рассеяния
    fig = make_subplots(rows=1, 
                        cols=2, 
                        subplot_titles=['Продажи-Пользователи', 'Продажи-Критики'], 
                        x_title='Продажи', 
                        y_title='Рейтинг')
    
    fig.add_trace(go.Scatter(x=games_data.query('platform == @platform')['total_sales'], 
                             y=games_data.query('platform == @platform')['user_score'], 
                             mode='markers', name='Оценки пользователей'
                            ), 
                  row=1, 
                  col=1
                 )
    
    fig.add_trace(go.Scatter(x=games_data.query('platform == @platform')['total_sales'], 
                             y=games_data.query('platform == @platform')['critic_score'], 
                             mode='markers', name='Оценки критиков'
                            ), 
                  row=1, 
                  col=2
                 )
    fig.update_layout(title=platform)
    return fig.show()

In [127]:
two_scatter_plots('PC')

Уже можно увидеть, что корреляция очень низка и в целом становится заметна только при высоких оценках. Расчитаем значения коэффициента Пирсона.

In [128]:
PC_data = games_data.query('platform == "PC"')
print(f'Коэффициент корреляции между оценками игроков и числом проданных копий: {PC_data["total_sales"].corr(PC_data["user_score"]) : .3}')
print(f'Коэффициент корреляции между оценками критиков и числом проданных копий: {PC_data["total_sales"].corr(PC_data["critic_score"]) : .3}')

Коэффициент корреляции между оценками игроков и числом проданных копий: -0.122
Коэффициент корреляции между оценками критиков и числом проданных копий:  0.237


Числовые показатели корреляции тоже говорят о том, что корреляция между оценками игроков и числом проданных копий отсутсвует полностью и практически отсутсвует между оценками критиками и проданными копиями.

Построим аналогичные диаграммы еще для нескольких платформ: PS3, PSV и 3DS.

In [139]:
for platform in ['PS3', 'PSV', '3DS']:
    two_scatter_plots(platform)
    platform_data = games_data.query('platform == @platform')
    print(f'Коэффициент корреляции между оценками игроков и числом проданных копий: {platform_data["total_sales"].corr(platform_data["user_score"]) : .3}')
    print(f'Коэффициент корреляции между оценками критиков и числом проданных копий: {platform_data["total_sales"].corr(platform_data["critic_score"]) : .3}')

Коэффициент корреляции между оценками игроков и числом проданных копий:  0.00282
Коэффициент корреляции между оценками критиков и числом проданных копий:  0.331


Коэффициент корреляции между оценками игроков и числом проданных копий:  0.0314
Коэффициент корреляции между оценками критиков и числом проданных копий:  0.0945


Коэффициент корреляции между оценками игроков и числом проданных копий:  0.224
Коэффициент корреляции между оценками критиков и числом проданных копий:  0.321


Видно, что продажи сильнее зависят от оценок критиков, чем от оценок пользователей. Но даже это значение довольно низко. Можно в целом утверждать, что небольшая зависимость между оценками и продажами есть - чем выше оценка критиков, тем больше копий продано. Она становится заметна при высоких значениях оценок критиков. Но эта зависимость значительно отличается для разных платформ и полагаться только на нее не стоит.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Корреляция подсчитана верно

## Исследование влияния жанра на продажи

 <a id='genre'></a>

Изучим число игр распределенное по жанрам.

In [130]:
px.bar(games_data.pivot_table(index='genre', 
                              values='name', 
                              aggfunc='count').sort_values('name').reset_index(), 
       x='name', 
       y='genre', 
       height=400
      )

Тройку лидеров занимуют экшн, ролевые игры и приключенческие. Выпускать игры - это хорошо, но их и продавать нужно. Так что по продажам? Изучим медианные значения числа проданных копий по жанрам.

In [131]:
px.bar(games_data.pivot_table(index='genre', 
                              values='total_sales', 
                              aggfunc='median').sort_values('total_sales').reset_index(), 
       x='total_sales', 
       y='genre', 
       height=400
      )

Видно, что несмотря, на большое количество игр жанра Action или Adventure продажи у них далеко не на высоте.

Шутеры, спортивные игры и платформеры занимают первые три места. Хотя медианное значние и довольно скромно - оно дает понимание в каком жанре игры продаются бóльшими тиражами.

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Общие продажи - плохая метрика для поиска наиболее прибыльного жанра. За высокими показателями общих продаж может скрываться множество мелких игр с низкими продажами. Или 2-3 звезды и куча провалов. Лучше найти жанр, где игры стабильно приносят высокий доход - для этого стоит рассмотреть средние или медианные продажи

# Портреты пользователей

<a id='regions_features'></a> 

## Самые популярные платформы в по регионам

In [132]:
region_portrait_platform = games_data.pivot_table(index='platform', values=['na_sales', 'eu_sales', 'jp_sales'], aggfunc='sum')
print('\nСеверная Америка')
na_data_platform = pd.concat([(region_portrait_platform['na_sales'] / region_portrait_platform['na_sales'].sum() * 100).
           sort_values(ascending=False), region_portrait_platform['na_sales'].
           sort_values(ascending=False).
           head()], axis=1).reset_index().head()
na_data_platform.columns = ['platform', 'na_sales_%', 'na_sales']
display(na_data_platform)
print('\nЕвропа')
eu_data_platform = pd.concat([(region_portrait_platform['eu_sales'] / region_portrait_platform['eu_sales'].sum() * 100).
           sort_values(ascending=False), region_portrait_platform['eu_sales'].
           sort_values(ascending=False).
           head()], axis=1).reset_index().head()
eu_data_platform.columns = ['platform', 'eu_sales_%', 'eu_sales']
display(eu_data_platform)
print('\nЯпония')
jp_data_platform = pd.concat([(region_portrait_platform['jp_sales'] / region_portrait_platform['jp_sales'].sum() * 100).
           sort_values(ascending=False), region_portrait_platform['jp_sales'].
           sort_values(ascending=False).
           head()], axis=1).reset_index().head()
jp_data_platform.columns = ['platform', 'jp_sales_%', 'jp_sales']
display(jp_data_platform)


Северная Америка


,platform,na_sales_%,na_sales
0,X360,23.698326,140.05
1,PS4,18.400257,108.74
2,PS3,17.493274,103.38
3,XOne,15.757145,93.12
4,3DS,9.359189,55.31



Европа


,platform,eu_sales_%,eu_sales
0,PS4,27.838286,141.09
1,PS3,21.084409,106.86
2,X360,14.703445,74.52
3,XOne,10.179156,51.59
4,3DS,8.413243,42.64



Япония


,platform,jp_sales_%,jp_sales
0,3DS,45.586250,87.79
1,PS3,18.324852,35.29
2,PSV,10.925330,21.04
3,PS4,8.287465,15.96
4,WiiU,6.755634,13.01


Составы пяти самых популярных платформ для Европы и Америки одинаковы, различаются места в первоц тройке. Для всех трех регионов первые позиции разные, но для Америки и Европы процент продаж на первую позицию довольно близок. А в Японии есть очевидный лидер - 3DS, который в Европе и в Америке занимается только 5-е место.
Для Японии состав изменился, вместо XOne и X360 - PSV и WiiU. Почему-то в Японии не любят играть на иксбоксе.

<a id='genre_features'></a> 

## Самые популярные жанры по регионам

In [133]:
region_portrait_genre = games_data.pivot_table(index='genre', values=['na_sales', 'eu_sales', 'jp_sales'], aggfunc='median')
region_portrait_genre.columns = ['eu_sales_median', 'jp_sales_median', 'na_sales_median']
print('\nСеверная Америка')
na_data_genre = region_portrait_genre['na_sales_median'].sort_values(ascending=False).head().reset_index().head()
display(na_data_genre)
print('\nЕвропа')
eu_data_genre = region_portrait_genre['eu_sales_median'].sort_values(ascending=False).head().reset_index().head()
display(eu_data_genre)
print('\nЯпония')
jp_data_genre = region_portrait_genre['jp_sales_median'].sort_values(ascending=False).head().reset_index().head()
display(jp_data_genre)


Северная Америка


,genre,na_sales_median
0,Shooter,0.20
1,Platform,0.09
2,Sports,0.08
3,Fighting,0.05
4,Racing,0.04



Европа


,genre,eu_sales_median
0,Shooter,0.17
1,Racing,0.09
2,Platform,0.08
3,Sports,0.05
4,Simulation,0.03



Япония


,genre,jp_sales_median
0,Role-Playing,0.06
1,Fighting,0.03
2,Misc,0.02
3,Action,0.01
4,Adventure,0.01


По жанрам наблюдается похожая картина: Америка и Европа имеют похожие наборы наиболее продаваемых жанров, а вот в Японии из этого набора только Fighting - да и тот популярен только в Америке, а первое место по продпжпм занимают ролевые компьютерные игры. Это необходимо учитывать, при развития продаж в конкретного регионе.

## Влияние рейтинга ESRB

In [134]:
rating_salse = games_data.pivot_table(index='rating', values=['na_sales', 'eu_sales', 'jp_sales', 'total_sales'], aggfunc='sum').reset_index()
rating_salse

,rating,eu_sales,jp_sales,na_sales,total_sales
0,E,113.03,28.33,114.37,287.17
1,E10+,55.37,8.19,75.70,155.49
2,M,193.96,21.20,231.57,510.11
3,T,52.96,26.02,66.02,162.94
4,unknown,91.50,108.84,103.31,330.82


In [135]:
fig = go.Figure()
fig.add_scatter(x=rating_salse['rating'], y=rating_salse['eu_sales'], name='Европа')
fig.add_scatter(x=rating_salse['rating'], y=rating_salse['na_sales'], name='Северная Америка')
fig.add_scatter(x=rating_salse['rating'], y=rating_salse['jp_sales'], name='Япония')
fig.update_yaxes(title='sales')
fig.update_xaxes(title='rating')

На основе имеющихся данных можно сказать, что рейтинг ESRB практически одинаково влияет на продажи игр в каждом регионе. Кривая для каждого региона имеет одну и туже форму, но разные объемы продаж - это может быть связано в первую очередь c различиями в численности населения и с экономическими особенностями. Общая картина все равно остается одинаковой для каждого региона.

Но особое внимание вызывает рейтинг T. В то время как для Евроопы и Америки наблюдается сильное снижение продаваемых игр с рейтингом T относительно игр с рейтингом M, в Японии наоборот, игры с рейтингом T даже популярнее, чем рейтинг M.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Все верно в этом пункте

# Проверка гипотез

## Гипотеза об одинаковости средних пользовательских оценок XboxOne и PC

Нулевой гипотезой будет являться то, что средние пользовательские оценки пользователей Xbox One и PC одинаковые, а альтернативной - то что они различаются.

In [136]:
alpha = 0.05


user_xbox_one_data = games_data.query('platform == "XOne"')['user_score'].dropna().values
user_PC_data = PC_data['user_score'].dropna().values
results = st.ttest_ind(user_xbox_one_data, user_PC_data, equal_var=False)

print('р-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

р-значение: 0.45802556131545946
Не получилось отвергнуть нулевую гипотезу


Проведя t-тест на равеснтво средних двух совокупностей отвергнуть гипотезу о равенстве средних оценок для XboxOne и PC не удалось. Это не доказывает гипотезу о равенстве этих средних, лишь говорит о том что при таком критическом уровне значимости отличия между ними статистически незначимы.

## Гипотеза о различии средних пользовательских оценок жанров Action и Sports

Проверяем мы факт их различия, но отталкиваться все равно будем от равенства.
Нулевой гипотезой будет являться то, что средние пользовательские оценки жанров Action и Sports равны, а альтернативной - то что они различаются.

In [137]:
alpha = 0.05


user_action_data = games_data.query('genre == "Action"')['user_score'].dropna().values
user_sports_data = games_data.query('genre == "Sports"')['user_score'].dropna().values
results = st.ttest_ind(user_action_data, user_sports_data, equal_var=False)

print('р-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

р-значение: 1.0495839755516034e-22
Отвергаем нулевую гипотезу


По итогам t-теста с критическим уровнем значимости 0.05 мы можем отвергнуть гипотезу о том, что оценки пользователей для жанров Action и Sports одинаковы. Отличия между ними значительны (в рамках выбранного уровня критической значимости).

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
     
Вывод стоит исправить - уровень значимости в выводе 0.01, а в коде 0.05

Для обоих вышеприведенных гипотез использовался t-тест, так как он позволяет сравнить две генеральные совокупности (срендние оценки пользователей) по выборке из них. Мы не обладаем всеми данными касательно средних оценок пользователей - более половины данных в этом столбце отсутсвует, поэтому имеющиеся данные - это лишь часть генеральной совокупности, именно с помощью этиой выборки и проводится сравнение средних пользовательских оценок в различных жанрах и платформах.

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 

Нужно сказать, почему выбран именно такой стат тест

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>
    
Уровень доверия 99 % чрезмерно велик. Такой уровень используется только в таких науках, где ошибка очень дорого стоит: медицина, химия и т.д., где мы не можем рисковать вообще, мы должны сотни раз перепроверить, что мы можем отказаться от нулевой гипотезы. В общественных науках (в т.ч. и в экономике), в бизнесе мы можем рисковать. И даже скажу более, в бизнесе нам надо рисковать для развития! Поэтому классический уровень доверия тут 95 %. Подробнее можешь почитать об этом тут: https://allatambov.github.io/psms/pdf/hypo-test.pdf

# Выводы

Для повышения качества результатов исследования полученные данные были предобработаны, учищены от пропусков по возможности с минимальными потерями данных.

Проведено многостороннее исследование, с целью определения наиболее [перспективных](#perspective_platforms) платформ на 2017 год. Такими платформами оказались:

* 3DS
* PS3
* PS4

Но я бы рекомендовал не упускать из вида и такие платформы как XOne, PC, PCV, они тоже все еще актуальны, а PC - это самая долгоживущая платформа. Поэтому нужно сделать упор на продвижение игр для этих платформ. Обязательно учитывая [особенности регионов](#regions_features) - в разных регионах люди пользуются разными платформами.

Так же в исследовании уделялось внимание наиболее популярным [жанрам](#genre). Наиболлее продаваемыми оказались игры жанра Shooter, Platform и Sports - поэтому лучше делать упор на продвижении игр именно этих жанров, но с оговорками. Необходимо иметь в виду, что регионы имеют [свои особенности](#genre_features), например в Японии огромным спросом пользуются компьютерные ролевые игры, а вот тройка вышеперечисленных там совсм не в почете. Поэтому необходимо проводить разные рекламные компании в разных регионах.


Была обнаружена небольшая зависимость числа проданных копий от оценок критиков - для разных платформ она выражена с разной силой. Чем выше оценки критиков - тем вероятнее игра будет продана большим тиражом, но к этим данным необходимо относиться осторожно, так как данные об оценках критиках в датасете были далеко не полные.

Были проверены две гипотезы о равенстве средних пользовательских оценок на платформах Xbox One и PC - была отвергнута, вероятно пользоатели этих платформ по-разному оценивают игры; и о различии пользовательских оценок жанров Action и Sports - наиболее популярных во всех трех исследуемых регионах - доказать эту гипотезу нет возможности, можно лишь попробовать отвергнуть гипотезу о равенстве этих средних оценок. Но этого сделать не удлаось, Возможно на большем объеме данных получится иной результат.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
Хороший общий вывод, молодец! В идеале можно дать четкие рекомендации бизнесу / заказчику

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Отличный проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев и рассуждений. Также очень хорошее оформление проекта. Сделано все именно так, как требуется. 

Что обязательно стоит доработать: написать, почему выбран именно такой статистический критерий для проверки гипотез; посмотреть иначе на самые популярные / прибыльные жанры; изменить актуальный срок для прогнозирования; доработать пункт с подготовкой данных (иначе обработать tbd значения)

Также я оставил несколько желтых комментариев. Думаю, и они будут тебе полезны.

Буду ждать доработок :) Удачи!

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌ Вторая итерация </b>

Все еще стоит изменить актуальный срок для прогнозирования.

Также обрати внимание на еще один желтый комментарий в пункте проверки гипотез. 

Все остальные пункты исправлены, молодец

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Третья итерация </b>

Все ошибки исправлены, проект стал еще лучше, принимаю. Успехов в дальнейшей учебе!